# 🦙 LLaMA-3 Fine-Tuning on Custom Dataset using Unsloth + LoRA

This notebook demonstrates how to fine-tune **Meta’s LLaMA-3 8B** model on a custom instruction dataset using the [Unsloth](https://github.com/unslothai/unsloth) library and [LoRA](https://arxiv.org/abs/2106.09685) adapters for parameter-efficient training.

Supports 4-bit quantization and runs efficiently on consumer-grade GPUs.

In [ ]:
%%capture
import torch

major_version, minor_version = torch.cuda.get_device_capability()

!pip install "unsloth[colab-new] @git+https://github.com/unslothai/unsloth.git"

if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

**2. Load LLaMA-3 8B Model with Unsloth**

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048         # Maximum sequence length for training
dtype = None                  # Let Unsloth auto-select precision
load_in_4bit = True           # Enable 4-bit quantized loading

# Supported pre-quantized models
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


**3. Inject LoRA Adapters (Low-Rank Adaptation)**

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA rank (commonly 8–64)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,             # 0 = optimal memory use
    bias = "none",                # Don't adapt biases for efficiency
    use_gradient_checkpointing = "unsloth",  # Reduces memory usage for long contexts
    random_state = 3407,
    use_rslora = False,           # Disable rank-stabilized LoRA
    loftq_config = None,          # Disable LoftQ quantization fine-tuning
)


**4. Format Instruction Dataset (Alpaca Format)**

In [ ]:
# Alpaca-style prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Get EOS token
EOS_TOKEN = tokenizer.eos_token

# Format each sample into a single prompt string
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }


<a name="Train"></a>
## Model Training: Fine-Tune with TRL's SFTTrainer

We fine-tune the model using Hugging Face's TRL `SFTTrainer`, a simple interface for supervised fine-tuning.  
Documentation: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer)

We use a small number of steps (`max_steps=60`) to keep training fast for demonstration.  
For proper fine-tuning, you can set `num_train_epochs=1` and remove `max_steps`.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,

    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

##  Check GPU Status and Memory Usage
Monitor GPU availability and memory stats to assess resource usage during training.


In [ ]:
import torch

In [ ]:
if torch.cuda.is_available():
    gpu_stats = torch.cuda.get_device_properties(0)
    allocated_gpu_memory = round(torch.cuda.memory_allocated(0) / 1024 / 1024 / 1024, 3)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
    print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
    print(f"{allocated_gpu_memory} GB of memory currently in use.")
else:
    print("CUDA is not available on this system.")

In [ ]:
trainer_stats = trainer.train()

###  Memory Stats After Training
Calculate peak reserved memory and estimate how much was used for LoRA fine-tuning.


In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - allocated_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

# Print runtime and memory stats
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime'] / 60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")


<a name="Inference"></a>
##  Inference: Generate Output from the Fine-Tuned Model

We run inference on the fine-tuned model using a simple instruction.  
You can change the prompt and input values below.


In [ ]:
# Prepare the model for inference (optimized for generation)
FastLanguageModel.for_inference(model)

# Define the instruction and input using the Alpaca-style prompt
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Continue the Fibonacci sequence.",  # Instruction
            "1, 1, 2, 3, 5, 8",                 # Input context
            "",                                 # Leave output blank
        )
    ],
    return_tensors="pt"
).to("cuda")

# Generate output tokens
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

tokenizer.batch_decode(outputs)


Use a `TextStreamer` to get real-time output from the model instead of waiting for full generation:


In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.",
        "1, 1, 2, 3, 5, 8",
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

### Saving and Loading Fine-tuned Models

You have a few options for saving and loading your fine-tuned model. You can save just the LoRA adapters or merge them with the base model for different use cases.

*   **Saving LoRA Adapters:** To save only the LoRA adapters (which are much smaller than the full model), you can use Hugging Face's `push_to_hub` to save to your Hugging Face repository or `save_pretrained` for a local save.

**Note:** Saving only the LoRA adapters means you'll need the original base model to use them for inference. If you want to save the full model in 16-bit or GGUF format, please refer to the sections below.

In [ ]:
model.save_pretrained("lora_model")

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?",
        "",
        "",
    )
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
if False:
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model",
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

# **GGUF Model Saving for llama.cpp**
Models can now be saved directly in GGUF format with native support. We auto-clone llama.cpp and default to q8_0 quantization. To save locally, use save_pretrained_gguf; to upload to Hugging Face, use push_to_hub_gguf.

Quantization options (full list):

*   q8_0: Fast, higher memory use
*   q4_k_m: Recommended, mixes Q6_K + Q4_K
*   q5_k_m: Recommended, mixes Q6_K + Q5_K


In [ ]:
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")